In [1]:
import QuantLib as ql

today = ql.Date().todaysDate()
ql.Settings.instance().evaluationDate = today

In [2]:
from src.utils.data_gathering import options_gathering, column_helper
from src.volatility.vol_surfaces import Black_Variance_Surface
from src.volatility.implied_volatilities import get_USImpliedVol

df = options_gathering(
    "AAPL",
    expiry_start_date="2025-05-25",
    expiry_end_date="2030-12-01",
    option_type="call",
    observed_vol=0.22,
    min_strike=200,
    max_strike=250,
)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,currency,Option Type,Ticker,expiration,date,time_to_maturity,underlying_price,dividend_yield,risk_free_rate,observed_vol
0,AAPL250620C00200000,2025-03-18 19:58:31+00:00,200.0,21.87,0.0,0.0,0.0,0.0,55.0,15940,...,USD,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22
1,AAPL250620C00205000,2025-03-18 19:53:59+00:00,205.0,18.50,0.0,0.0,0.0,0.0,44.0,7076,...,USD,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22
2,AAPL250620C00210000,2025-03-18 19:55:42+00:00,210.0,15.21,0.0,0.0,0.0,0.0,137.0,14078,...,USD,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22
3,AAPL250620C00215000,2025-03-18 19:40:59+00:00,215.0,12.50,0.0,0.0,0.0,0.0,775.0,12395,...,USD,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22
4,AAPL250620C00220000,2025-03-18 19:50:09+00:00,220.0,9.85,0.0,0.0,0.0,0.0,899.0,12486,...,USD,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,AAPL271217C00230000,2025-03-18 13:49:13+00:00,230.0,40.35,0.0,0.0,0.0,0.0,1.0,229,...,USD,CALL,AAPL,2027-12-17,2025-03-19 12:55:45.358335,2.743326,212.690002,0.004689,0.041094,0.22
119,AAPL271217C00235000,2025-03-18 16:21:51+00:00,235.0,37.00,0.0,0.0,0.0,0.0,2.0,30,...,USD,CALL,AAPL,2027-12-17,2025-03-19 12:55:45.358335,2.743326,212.690002,0.004689,0.041094,0.22
120,AAPL271217C00240000,2025-03-18 18:49:25+00:00,240.0,34.57,0.0,0.0,0.0,0.0,14.0,585,...,USD,CALL,AAPL,2027-12-17,2025-03-19 12:55:45.358335,2.743326,212.690002,0.004689,0.041094,0.22
121,AAPL271217C00245000,2025-03-14 15:05:11+00:00,245.0,32.76,0.0,0.0,0.0,0.0,NaN,19,...,USD,CALL,AAPL,2027-12-17,2025-03-19 12:55:45.358335,2.743326,212.690002,0.004689,0.041094,0.22


In [3]:
df_cop = df.rename(columns=column_helper(vol_col="observed_vol"))
df_cop = get_USImpliedVol(df_cop, end_col="calculated_vol")
df_cop

Implied volatility calculation failed for 0.00% of values and was filled with interpolation


,contractSymbol,lastTradeDate,k,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,option_type,Ticker,exercise_date,date,time_to_maturity,u,div,r,sigma,calculated_vol
0,AAPL250620C00200000,2025-03-18 19:58:31+00:00,200.0,21.87,0.0,0.0,0.0,0.0,55.0,15940,...,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22,0.323874
1,AAPL250620C00205000,2025-03-18 19:53:59+00:00,205.0,18.50,0.0,0.0,0.0,0.0,44.0,7076,...,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22,0.314943
2,AAPL250620C00210000,2025-03-18 19:55:42+00:00,210.0,15.21,0.0,0.0,0.0,0.0,137.0,14078,...,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22,0.301782
3,AAPL250620C00215000,2025-03-18 19:40:59+00:00,215.0,12.50,0.0,0.0,0.0,0.0,775.0,12395,...,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22,0.295875
4,AAPL250620C00220000,2025-03-18 19:50:09+00:00,220.0,9.85,0.0,0.0,0.0,0.0,899.0,12486,...,CALL,AAPL,2025-06-20,2025-03-19 12:55:45.358335,0.251882,212.690002,0.004689,0.041094,0.22,0.284288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,AAPL271217C00230000,2025-03-18 13:49:13+00:00,230.0,40.35,0.0,0.0,0.0,0.0,1.0,229,...,CALL,AAPL,2027-12-17,2025-03-19 12:55:45.358335,2.743326,212.690002,0.004689,0.041094,0.22,0.279178
119,AAPL271217C00235000,2025-03-18 16:21:51+00:00,235.0,37.00,0.0,0.0,0.0,0.0,2.0,30,...,CALL,AAPL,2027-12-17,2025-03-19 12:55:45.358335,2.743326,212.690002,0.004689,0.041094,0.22,0.268249
120,AAPL271217C00240000,2025-03-18 18:49:25+00:00,240.0,34.57,0.0,0.0,0.0,0.0,14.0,585,...,CALL,AAPL,2027-12-17,2025-03-19 12:55:45.358335,2.743326,212.690002,0.004689,0.041094,0.22,0.263649
121,AAPL271217C00245000,2025-03-14 15:05:11+00:00,245.0,32.76,0.0,0.0,0.0,0.0,NaN,19,...,CALL,AAPL,2027-12-17,2025-03-19 12:55:45.358335,2.743326,212.690002,0.004689,0.041094,0.22,0.263059


In [4]:
bsv = Black_Variance_Surface(df_cop, vol_col="calculated_vol")
bsv.plot_vol_surface(200, 250)